<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/ChemNLP_TitleToAbstract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate rouge_score jarvis-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 k

In [2]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import torch
import math
from tqdm import tqdm
import time
import evaluate
from collections import defaultdict
from jarvis.db.jsonutils import dumpjson
import random
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

random_seed = 123
torch.manual_seed(random_seed)
random.seed(0)
np.random.seed(random_seed)
torch.backends.cudnn.deterministic = True

rouge_score = evaluate.load("rouge")

# import torch
# torch.cuda.is_available = lambda : False

tqdm.pandas()
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
  

In [4]:

# prompt="Nonuniform superconductivity and Josephson effect in conical ferromagnet can be described as"
model_checkpoint = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
def generate_text(prompt="What is a superconductor?",max_new_tokens=250,model_checkpoint = "gpt2-medium"):
    # tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
    # model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
    inputs = tokenizer(prompt, return_tensors="pt")
    # model = AutoModelForCausalLM.from_pretrained(checkpoint)
    # outputs = model.generate(**inputs, do_sample=True)
    outputs = model.generate(
        **inputs, do_sample=True, max_new_tokens=max_new_tokens
    )
    out = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return (out[0].replace('\n',' '))

In [5]:
p = generate_text()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
p

In [6]:
import os
if not os.path.exists('jarvis_leaderboard'):
  !git clone https://github.com/usnistgov/jarvis_leaderboard.git
os.chdir('jarvis_leaderboard')
!pip install -e .

Cloning into 'jarvis_leaderboard'...
remote: Enumerating objects: 37380, done.
remote: Counting objects: 100% (6180/6180), done.
remote: Compressing objects: 100% (655/655), done.
remote: Total 37380 (delta 3150), reused 5969 (delta 3079), pack-reused 31200
Receiving objects: 100% (37380/37380), 281.76 MiB | 31.92 MiB/s, done.
Resolving deltas: 100% (19318/19318), done.
Updating files: 100% (2377/2377), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/jarvis_leaderboard
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.3 MB/s eta 0:00:00
  

In [7]:
!ls jarvis_leaderboard/benchmarks/AI/TextGen/arxiv_gen_text.json.zip

jarvis_leaderboard/benchmarks/AI/TextGen/arxiv_gen_text.json.zip


In [8]:
import json,zipfile
fname = 'jarvis_leaderboard/benchmarks/AI/TextGen/arxiv_gen_text.json.zip'
temp = 'arxiv_gen_text.json'
zp = zipfile.ZipFile(fname)
train_test = json.loads(zp.read(temp))

In [9]:
from jarvis.db.figshare import data
import pandas as pd
arxiv_summary = data('arxiv_summary')
df = pd.DataFrame(arxiv_summary)
df = df.drop_duplicates()

Obtaining arxiv summary cond.mat dataset 137927...
Reference:https://github.com/usnistgov/chemnlp


100%|██████████| 48.9M/48.9M [00:02<00:00, 19.3MiB/s]


Loading the zipfile...
Loading completed.


In [10]:
%%time

from tqdm import tqdm
info = {}
for i,j in tqdm(train_test['test'].items()):
  title=df[df['id']==i]['text'].values[0]
  actual_abstract =df[df['id']==i]['ctext'].values[0]
  prompt = title+' can be described as '
  #prompt = 'Write an abstract on the title '+ prompt
  pred = generate_text(prompt)
  info[i] = pred
  #break

100%|██████████| 98/98 [1:04:15<00:00, 39.35s/it]

CPU times: user 1h 3min 37s, sys: 5.95 s, total: 1h 3min 43s
Wall time: 1h 4min 15s


In [11]:
from jarvis.db.jsonutils import dumpjson
dumpjson(data=info,filename='textgen.json')

In [18]:
mem=[]
for i,j in info.items():
  info1={}
  info1['id']=str(i)
  info1['prediction']=j
  mem.append(info1)
  

In [19]:
import pandas as pd
dff=pd.DataFrame(mem)
dff

,id,prediction
0,1812.09136,Nonuniform superconductivity and Josephson eff...
1,cond-mat/0408037,Further detailing of the Bose-Einstein negativ...
2,cond-mat/0506765,Tunable critical current for a vortex pinned b...
3,cond-mat/0105402,"MgB_2 under pressure: phonon calculations, Ram..."
4,cond-mat/0401637,Dissipative Currents in Superfluid 3He Weak Li...
...,...,...
93,cond-mat/0306416,Doping dependent time-reversal symmetric nonli...
94,1510.08177,Electronic structure of a superconducting topo...
95,2008.07814,Theory of Supercurrent in Superconductors can ...
96,1205.5540,Evidence of strong correlations at the van Hov...


In [20]:
dff.to_csv('AI-TextGen-text-arxiv_gen-test-rouge.csv',index=False)



In [22]:
!zip AI-TextGen-text-arxiv_gen-test-rouge.csv.zip AI-TextGen-text-arxiv_gen-test-rouge.csv

  adding: AI-TextGen-text-arxiv_gen-test-rouge.csv (deflated 63%)
